In [8]:
import numpy as np
import matplotlib.pyplot as plt
import math
import torch
import torch.optim as optim
from sklearn.model_selection import KFold
from preprocessing import *
from model import *
from trainSTP import *
from model import STPGSR
from types import SimpleNamespace
import time


In [4]:
if torch.cuda.is_available():
    num_devices = torch.cuda.device_count()
    for i in range(num_devices):
        device = torch.cuda.device(i)
        total_mem = torch.cuda.get_device_properties(i).total_memory / 1024**3  # Convert to GB
        allocated_mem = torch.cuda.memory_allocated(i) / 1024**3  # Convert to GB
        free_mem = total_mem - allocated_mem
        
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"Total Memory: {total_mem:.1f}GB")
        print(f"Allocated Memory: {allocated_mem:.1f}GB")
        print(f"Free Memory: {free_mem:.1f}GB")
        
        if free_mem < 8:
            print(f"Warning: GPU {i} has less than 8GB of free VRAM!")
        else:
            print(f"Using GPU {i} with {free_mem:.1f}GB free VRAM")
            break 
    device = torch.device(f"cuda:{0}")
else:
    print("Warning: No CUDA devices available - running on CPU only")
    device = torch.device("cpu")

GPU 0: NVIDIA RTX A6000
Total Memory: 44.6GB
Allocated Memory: 0.0GB
Free Memory: 44.6GB
Using GPU 0 with 44.6GB free VRAM


In [5]:
def anti_vec(vec):
    N = math.ceil(math.sqrt(vec.shape[-1] * 2))
    adj = torch.zeros((vec.shape[0], N, N), dtype=vec.dtype)
    row_idx, col_idx = torch.triu_indices(N, N, offset=1)
    row_idx = (N - 1 - row_idx).flip(dims=[0])
    col_idx = (N - 1 - col_idx).flip(dims=[0])
    adj[:, row_idx, col_idx] = vec
    adj[:, col_idx, row_idx] = vec
    return adj

In [6]:
training_data = np.loadtxt('../lr_train.csv', delimiter=',', skiprows=1)
training_data_adj = anti_vec(torch.from_numpy(training_data))

training_label = np.loadtxt('../hr_train.csv', delimiter=',', skiprows=1)
training_label_adj = anti_vec(torch.from_numpy(training_label))

In [13]:
n = math.ceil(math.sqrt(training_data.shape[-1] * 2))
n_prime =  math.ceil(math.sqrt(training_label.shape[-1] * 2))

args = {
    "lr_dim": 160,
    "hr_dim": 268,
    "hidden_dim": 268,
    "lr": 0.005,
    "epochs": 100,
    "padding": 26,
    "device": device,
    "batch_size": len(training_data) // 10
}
args = SimpleNamespace(**args)
model = STPGSR(lr_n = 160, hr_n = 268).to(device)
state_dict = torch.load(f"./model_{0}/model.pth")
model.load_state_dict(state_dict)

<All keys matched successfully>

In [20]:
total_time = 0.0
max_memory_use = 0.0
max_memory_reserve = 0.0
for x, y in zip(training_data_adj, training_label_adj):
    torch.cuda.reset_peak_memory_stats()
    initial_memory = torch.cuda.memory_allocated()
    x = torch.tensor(x, dtype=torch.float32).to(device)
    source_g = create_pyg_graph(x, args.lr_dim).to(device)
    start_time = time.time()
    pred, model_target = model(source_g, y)
    total_time += time.time() - start_time
    max_memory_use = max(max_memory_use, torch.cuda.max_memory_allocated() - initial_memory)
total_time /= len(training_data_adj)
print(f"Max Memory Allocated: {max_memory_use / 1024**2:.2f} MB")
print(f"Average Time per Sample: {total_time} s")

/tmp/ipykernel_2768385/3990947519.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32).to(device)


Max Memory Allocated: 681.70 MB
Average Time per Sample: 0.008518761503482294 s


In [23]:
model = model.to('cpu')
total_time = 0.0
for x, y in zip(training_data_adj, training_label_adj):
    x = torch.tensor(x, dtype=torch.float32)
    source_g = create_pyg_graph(x, args.lr_dim)
    start_time = time.time()
    pred, _ = model(source_g, y)
    total_time += time.time() - start_time
total_time /= len(training_data_adj)
print(f"Average Time per Sample: {total_time} s")

/tmp/ipykernel_2768385/3064790379.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)


Average Time per Sample: 0.5952840151187189 s
